# CVAE
A Conditional Variational Autoencoder (CVAE) is a type of neural network that combines the concepts of variational autoencoders (VAEs) and conditional models. It is used for generating new data samples that are conditioned on some input data. In a CVAE, both the encoder and decoder are conditioned on additional information, which allows the model to generate more specific and controlled outputs. This makes CVAEs particularly useful for tasks such as image generation, where the generated images need to adhere to certain conditions or labels.

## Imports

In [32]:
# !pip install torch torchvision

In [33]:
import torch
import torch.utils.data
from torch import nn, optim
from torch.nn import functional as F
from torchvision import datasets, transforms
from torchvision.utils import save_image

## cuda setup
for gpu

In [34]:
# device = torch.device("cuda")
# kwargs = {'num_workers': 1, 'pin_memory': True} 

for cpu

In [35]:
device = torch.device("cpu")
kwargs = {'num_workers': 1, 'pin_memory': False} 


# Model Definition

In [36]:
class CVAE(nn.Module):
    def __init__(self, feature_size, latent_size, class_size):
        super(CVAE, self).__init__()
        self.feature_size = feature_size
        self.class_size = class_size

        # encode
        self.fc1  = nn.Linear(feature_size + class_size, 400)
        self.fc21 = nn.Linear(400, latent_size)
        self.fc22 = nn.Linear(400, latent_size)

        # decode
        self.fc3 = nn.Linear(latent_size + class_size, 400)
        self.fc4 = nn.Linear(400, feature_size)

        self.elu = nn.ELU()
        self.sigmoid = nn.Sigmoid()

    def encode(self, x, c): # Q(z|x, c)
        '''
        x: (bs, feature_size)
        c: (bs, class_size)
        '''
        inputs = torch.cat([x, c], 1) # (bs, feature_size+class_size)
        h1 = self.elu(self.fc1(inputs))
        z_mu = self.fc21(h1)
        z_var = self.fc22(h1)
        return z_mu, z_var
    
    def reparameterize(self, mu, logvar):
        std = torch.exp(0.5*logvar)
        eps = torch.randn_like(std)
        return mu + eps*std

    def decode(self, z, c): # P(x|z, c)
        '''
        z: (bs, latent_size)
        c: (bs, class_size)
        '''
        inputs = torch.cat([z, c], 1) # (bs, latent_size+class_size)
        h3 = self.elu(self.fc3(inputs))
        return self.sigmoid(self.fc4(h3))

    def forward(self, x, c):
        mu, logvar = self.encode(x.view(-1, 28*28), c)
        z = self.reparameterize(mu, logvar)
        return self.decode(z, c), mu, logvar


In [37]:
def one_hot(labels, class_size):
    targets = torch.zeros(labels.size(0), class_size)
    for i, label in enumerate(labels):
        targets[i, label] = 1
    return targets.to(device)

# Reconstruction + KL divergence losses summed over all elements and batch
def loss_function(recon_x, x, mu, logvar):
    BCE = F.binary_cross_entropy(recon_x, x.view(-1, 784), reduction='sum')
    # see Appendix B from VAE paper:
    # Kingma and Welling. Auto-Encoding Variational Bayes. ICLR, 2014
    # https://arxiv.org/abs/1312.6114
    # 0.5 * sum(1 + log(sigma^2) - mu^2 - sigma^2)
    KLD = -0.5 * torch.sum(1 + logvar - mu.pow(2) - logvar.exp())
    return BCE + KLD

def train(epoch):
    model.train()
    train_loss = 0
    for batch_idx, (data, labels) in enumerate(train_loader):
        data, labels = data.to(device), labels.to(device)
        labels = one_hot(labels, 10)
        recon_batch, mu, logvar = model(data, labels)
        optimizer.zero_grad()
        loss = loss_function(recon_batch, data, mu, logvar)
        loss.backward()
        train_loss += loss.detach().cpu().numpy()
        optimizer.step()
        if batch_idx % 20 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader),
                loss.item() / len(data)))

    print('====> Epoch: {} Average loss: {:.4f}'.format(
          epoch, train_loss / len(train_loader.dataset)))

def test(epoch):
    model.eval()
    test_loss = 0
    with torch.no_grad():
        for i, (data, labels) in enumerate(test_loader):
            data, labels = data.to(device), labels.to(device)
            labels = one_hot(labels, 10)
            recon_batch, mu, logvar = model(data, labels)
            test_loss += loss_function(recon_batch, data, mu, logvar).detach().cpu().numpy()
            if i == 0:
                n = min(data.size(0), 5)
                comparison = torch.cat([data[:n],
                                      recon_batch.view(-1, 1, 28, 28)[:n]])
                save_image(comparison.cpu(),
                         'reconstruction_' + str(f"{epoch:02}") + '.png', nrow=n)

    test_loss /= len(test_loader.dataset)
    print('====> Test set loss: {:.4f}'.format(test_loss))

## Hyper Parameters

In [38]:
batch_size = 64
latent_size = 20
epochs = 30

# Load data

In [39]:
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('./data', train=True, download=True,
                   transform=transforms.ToTensor()),
    batch_size=batch_size, shuffle=True, **kwargs)

test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('./data', train=False, transform=transforms.ToTensor()),
    batch_size=batch_size, shuffle=False, **kwargs)

# Main

In [40]:
model = CVAE(28*28, latent_size, 10).to(device)

optimizer = optim.Adam(model.parameters(), lr=1e-3)

for epoch in range(1, epochs + 1):
        train(epoch)
        test(epoch)
        with torch.no_grad():
            c = torch.eye(10, 10).to(device)
            sample = torch.randn(10, latent_size).to(device)
            sample = model.decode(sample, c).cpu()
            save_image(sample.view(10, 1, 28, 28),
                       'sample_' + str(f"{epoch:02}") + '.png')

Train Epoch: 1 [0/60000 (0%)]	Loss: 549.863037
Train Epoch: 1 [1280/60000 (2%)]	Loss: 239.116058
Train Epoch: 1 [2560/60000 (4%)]	Loss: 220.010651
Train Epoch: 1 [3840/60000 (6%)]	Loss: 184.171661
Train Epoch: 1 [5120/60000 (9%)]	Loss: 179.036407
Train Epoch: 1 [6400/60000 (11%)]	Loss: 174.754715
Train Epoch: 1 [7680/60000 (13%)]	Loss: 164.586426
Train Epoch: 1 [8960/60000 (15%)]	Loss: 165.547256
Train Epoch: 1 [10240/60000 (17%)]	Loss: 159.677856
Train Epoch: 1 [11520/60000 (19%)]	Loss: 162.557068
Train Epoch: 1 [12800/60000 (21%)]	Loss: 148.429825
Train Epoch: 1 [14080/60000 (23%)]	Loss: 147.895493
Train Epoch: 1 [15360/60000 (26%)]	Loss: 152.831635
Train Epoch: 1 [16640/60000 (28%)]	Loss: 147.057175
Train Epoch: 1 [17920/60000 (30%)]	Loss: 144.950974
Train Epoch: 1 [19200/60000 (32%)]	Loss: 135.773788
Train Epoch: 1 [20480/60000 (34%)]	Loss: 140.534882
Train Epoch: 1 [21760/60000 (36%)]	Loss: 138.966934
Train Epoch: 1 [23040/60000 (38%)]	Loss: 138.119324
Train Epoch: 1 [24320/60000 

In [41]:
import numpy as np

with torch.no_grad():
    class_index = 0
    c = torch.zeros(10, 10).to(device)
    c[:, class_index] = 1
    sample = torch.randn(10, latent_size).to(device)
    sample = model.decode(sample, c).cpu()
    save_image(sample.view(10, 1, 28, 28), 'sample_class_10.png')

    # import matplotlib.pyplot as plt

    # # Print the sample tensor as an image
    # plt.figure(figsize=(10, 1))
    # for i in range(10):
    #     plt.subplot(1, 10, i + 1)
    #     plt.imshow(sample[i].view(28, 28).cpu().numpy(), cmap='gray')
    #     plt.axis('off')
    # plt.show()

In [42]:
torch.save(model.state_dict(), 'cvae_model.pth')

# Load Model

In [43]:
model = CVAE(28*28, latent_size, 10).to(device)
model.load_state_dict(torch.load('cvae_model.pth'))
model.eval()

/tmp/ipykernel_46337/3387257094.py:2: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load('cvae_model.pth'))


CVAE(
  (fc1): Linear(in_features=794, out_features=400, bias=True)
  (fc21): Linear(in_features=400, out_features=20, bias=True)
  (fc22): Linear(in_features=400, out_features=20, bias=True)
  (fc3): Linear(in_features=30, out_features=400, bias=True)
  (fc4): Linear(in_features=400, out_features=784, bias=True)
  (elu): ELU(alpha=1.0)
  (sigmoid): Sigmoid()
)

In [44]:
import matplotlib.pyplot as plt

with torch.no_grad():
    class_index = 0
    c = torch.zeros(10, 10).to(device)
    c[:, class_index] = 1
    sample = torch.randn(10, latent_size).to(device)
    sample = model.decode(sample, c).cpu()
    # save_image(sample.view(10, 1, 28, 28), 'sample_class_10.png')

    # # Print the sample tensor as an image
    plt.figure(figsize=(10, 1))
    for i in range(10):
        plt.subplot(1, 10, i + 1)
        plt.imshow(sample[i].view(28, 28).cpu().numpy(), cmap='gray')
        plt.axis('off')
    plt.show()

ModuleNotFoundError: No module named 'matplotlib'